# **AIDI-1010**


---



# **Contents:**
**Module A: QuTiP**

1.   Installing The Module (QuTiP)
2.   Defining Qubit
3.   Generating a Control Pulse
4.   Takeaways

**Module B: Qiskit**

1.   Installing The Module (Qiskit-MachineLearning)
2.   ML Experiment
3.   Offline Examples




# **Module A: QuTiP**
---



# **1 - Installing QuTiP (Google Colab)**

In [ ]:
#1.1 - Install Linux Dependencies & Module (20secs)
!pip install qutip

**Note: Restart Runtime & Re-Run below**



In [ ]:
#1.2 - Re-Check Linux Dependencies & Module (5-10secs)
!pip install qutip

# **2- Defining a Qubit**
Create a quntum object, and inspect it using different methods.

Simulate rotations on an N qubit system using the "Processor" class. Considering a single qubit, so N = 1 in our case. 

In [ ]:
#Credit: https://www.activestate.com/blog/emulating-quantum-computing-with-python/ & https://colab.research.google.com/github/jrjohansson/qutip-lectures/blob/master/Lecture-0-Introduction-to-QuTiP.ipynb#scrollTo=8Ow9I6ng6Rqb

#2.1 - Load Modules
from qutip import *
from qutip import sigmaz
from qutip.qip.device import Processor

In [ ]:
#2.2 - Create Quantum Object using Qobj class; 
#We will pass a python list as an argument to the class constructor. 
#Data in list is used to construct the matrix representation of the quantum objects, and the other properties of the quantum object is by default computed from the same data.

q = Qobj([[1], [0]])
q

In [ ]:
#2.2 - Inspecting the Quantum Object; the dimension, or composite Hilbert state space structure
q.dims

In [ ]:
#2.3 - Inspecting the Quantum Object; the shape of the matrix data representation
q.shape

In [ ]:
#2.4 - Inspecting the Quantum Object; the matrix data itself. in sparse matrix format. 
q.data

In [ ]:
#2.5 - Inspecting the Quantum Object; get the dense matrix representation
q.full()

In [ ]:
#2.6 - Inspecting the Quantum Object; some additional properties
q.isherm, q.type 

In [ ]:
#2.7 Create a processor in preparation of generating a control pulse
processor = Processor(N=1)

# **3- Generate Control Pulses**
To quantify how magnitude of noise influences the fidelity of superposition state of a qubit.
To add control pulses, we use "add_control" method. We will add two pulses, σz and σy - which will prepare the qubit in superposition state.

In [ ]:
#3.1 - Add two pulses to prepare qubit in superposition state
processor.add_control(0.5 * sigmaz(), targets=0, label="sigmaz")
processor.add_control(0.5 * sigmay(), targets=0, label="sigmay")

In [ ]:
#3.2 - Review characteristics of each pulse
for pulse in processor.pulses:
    pulse.print_info()


#The results shows "targets": [0], which means the target qubit - pulses are trying to interact with it. 
#Python shows the value of [0] meaning only one target and impact.
#tlist = time sequence of pulse
#coeff = strength of pulse
#Next step will define both tlist and coeff

In [ ]:
#3.3 - Define the pulses (tlist & coeff) properly
import numpy as np
from numpy import pi

processor.pulses[1].tlist = np.array([0., pi]) #time sequence defined from 0 to pi
processor.pulses[1].coeff = np.array([1.]) # pulse strength set to 1
for pulse in processor.pulses:
    pulse.print_info()

In [ ]:
#3.4 - To observe effect of coeff pulse on qubit - need to initialize the qubit in the '0' state

#define initial state
basis0 = basis(2, 0)
basis0

In [ ]:
#3.5 - Run the simulation for qubit to be in the '1' state
result = processor.run_state(init_state=basis0)
result.states[-1].tidyup(1.0e-5)

#qubit is now in the "1" state

# **4- Prepare Qubit in SuperPosition state**

In [ ]:
#4.1 - Preparing three pulses with rotations

#rotations to prepare superposition state
processor.pulses[0].coeff = np.array([1., 0., 1.])
processor.pulses[1].coeff = np.array([0., 1., 0.])
processor.pulses[0].tlist = np.array([0., pi/2., 2*pi/2, 3*pi/2])
processor.pulses[1].tlist = np.array([0., pi/2., 2*pi/2, 3*pi/2])

In [ ]:
#4.2 - Observing results
result = processor.run_state(init_state=basis(2, 1))
result.states[-1].tidyup(1e-5)

#if the array is 0.707 \\ 0.707j -- means its composed of both 0 & 1 state 

In [ ]:
#4.3 - Visualize results and superposition
b = Bloch()
b.add_states(result.states)
b.make_sphere()

#The initial state is 1, which corresponds to a vector pointing in the -z direction. 
#After the first pulse, the qubit maintains the same state (yellow vector). 
#The blue and red vectors correspond to the qubit state after the second and third pulse. 
#Continue further reading: https://www.activestate.com/blog/emulating-quantum-computing-with-python/

# **Module B: Qiskit**
---



# **1 - Installing Qiskit (Google Colab)**

In [ ]:
#1.1 - Install Linux Dependencies & Module (20secs)
!pip install qiskit-machine-learning

**Note: Restart Runtime & Re-Run below**



In [ ]:
#1.2 - Re-Check Linux Dependencies & Module (5-10secs)
!pip install qiskit-machine-learning

# **2- Machine Learning Experiment using Qiskit**
https://github.com/Qiskit/qiskit-machine-learning

 
1.   Build: Design a quantum circuit(s) that represents the problem you are considering.
2.   Compile: Compile circuits for a specific quantum service, e.g. a quantum system or classical simulator.
3.   Run: Run the compiled circuits on the specified quantum service(s). These services can be cloud-based or local.
4.   Analyze: Compute summary statistics and visualize the results of the experiments.



It has some classification algorithms such as QSVM and VQC (Variational Quantum Classifier), where this data can be used for experiments, and there is also QGAN (Quantum Generative Adversarial Network) algorithm.

In [ ]:
#2.1 - Load suppporting modules
from qiskit import BasicAer
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms.optimizers import COBYLA #numerical optimization method
from qiskit.circuit.library import TwoLocal #rotation layers and entaglement layers
from qiskit_machine_learning.algorithms import VQC #quantum algorithm 
from qiskit_machine_learning.datasets import wine #data
from qiskit_machine_learning.circuit.library import RawFeatureVector

In [ ]:
#2.2 - Load algorithm
seed = 1376
algorithm_globals.random_seed = seed

In [ ]:
#2.3 - Use Wine data set for training and test data
feature_dim = 4  # dimension of each data point
training_size = 12
test_size = 4

In [ ]:
#2.4 - Training features, training labels, test features, test labels as np.array,one hot encoding for labels
training_features, training_labels, test_features, test_labels = \
    wine(training_size=training_size, test_size=test_size, n=feature_dim)

feature_map = RawFeatureVector(feature_dimension=feature_dim)
ansatz = TwoLocal(feature_map.num_qubits, ['ry', 'rz'], 'cz', reps=3)
vqc = VQC(feature_map=feature_map,
          ansatz=ansatz,
          optimizer=COBYLA(maxiter=100),
          quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                            shots=1024,
                                            seed_simulator=seed,
                                            seed_transpiler=seed)
          )

In [ ]:
#2.5 - Fit the data into model (~1-2min)
vqc.fit(training_features, training_labels)

In [ ]:
#2.6 - Evaluate score of model (~1-2mins)
score = vqc.score(test_features, test_labels)
print('Testing accuracy: {:0.2f}'.format(score))

# **3 - Offline Examples**

In [ ]:
https://qiskit.org/documentation/machine-learning/tutorials